## Winter Olympics- medals per athlete
In this notebook we are going to join a dataset that contains the total number of athletes who have ever participated in any Winter Olympic game with another dataset that contains total medals achieved per country since 1960. 

In [1]:
import pandas as pd
df= pd.read_csv('athlete_events.csv')

In [2]:
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


### Becasue we are only interested in the Winter Olympics since 1960, we filter out all the unwanted columns and rows 

In [4]:
df = df[(df.Season =='Winter') & (df.Year >= 1960)]

In [5]:
# there is a total of 848 athletes
Total_athletes = df.groupby(["NOC", "Year"]).ID.unique().to_frame().reset_index()
Total_athletes.head()

,NOC,Year,ID
0,AHO,1988,"[18463, 29956]"
1,AHO,1992,"[18463, 27536]"
2,ALB,2006,[120779]
3,ALB,2010,[120779]
4,ALB,2014,[78179]


In [6]:
# use the lambda function to count unique ID, so we don't duplicates athletes 
Total_athletes['Total_athletes'] = Total_athletes['ID'].apply(lambda x: len(x))
Total_athletes =Total_athletes.drop(['ID'], axis = 1)
Total_athletes.head()

,NOC,Year,Total_athletes
0,AHO,1988,2
1,AHO,1992,2
2,ALB,2006,1
3,ALB,2010,1
4,ALB,2014,1


In [7]:
# we read our other dataset that contains total number of medals
df1 = pd.read_excel('Medals_converted.xlsx', sheet_name='Historic_Medalist')
df1.Year = df1.Year.astype(int)
df1['Total Medals'] = df1['Total Medals'].astype(int)
df1.head()

,Countries,Host_city,Year,Total Medals,NOC
0,Austria,Squaw Valley 1960,1960,6,AUT
1,Canada,Squaw Valley 1960,1960,4,CAN
2,Finland,Squaw Valley 1960,1960,8,FIN
3,France,Squaw Valley 1960,1960,3,FRA
4,Germany,Squaw Valley 1960,1960,8,GER


In [8]:
df =pd.merge(Total_athletes, df1, on=(['Year', 'NOC']), how='right')
df.head()

,NOC,Year,Total_athletes,Countries,Host_city,Total Medals
0,AUT,1960,26.0,Austria,Squaw Valley 1960,6
1,CAN,1960,44.0,Canada,Squaw Valley 1960,4
2,FIN,1960,48.0,Finland,Squaw Valley 1960,8
3,FRA,1960,26.0,France,Squaw Valley 1960,3
4,GER,1960,74.0,Germany,Squaw Valley 1960,8


In [9]:
# we save our merged dataframe
df = df.to_csv("merged_olympics.csv", index=False)

In [10]:
# we splitted in Excel year and host city in two columns
# and saved as a new csv file
df_final= pd.read_csv('merged_olympics2.csv')
df_final

,National_Olympic_Committees,Countries,year,Host_city,total_athletes,total-medals
0,AUT,Austria,1960,Squaw Valley,26.0,6
1,CAN,Canada,1960,Squaw Valley,44.0,4
2,FIN,Finland,1960,Squaw Valley,48.0,8
3,FRA,France,1960,Squaw Valley,26.0,3
4,GER,Germany,1960,Squaw Valley,74.0,8
...,...,...,...,...,...,...
217,SWE,Sweden,2018,PyeongChang,116.0,14
218,KOR,South Korea,2018,PyeongChang,122.0,17
219,JPN,Japan,2018,PyeongChang,124.0,13
220,CHN,People's Republic of China,2018,PyeongChang,80.0,9


In [11]:
# calculate the national rate per country of athletes per medals achieved
df_final['Athletes_per_medal'] = round(df_final.total_athletes / df_final['total-medals'], 2)
df_final

,National_Olympic_Committees,Countries,year,Host_city,total_athletes,total-medals,Athletes_per_medal
0,AUT,Austria,1960,Squaw Valley,26.0,6,4.33
1,CAN,Canada,1960,Squaw Valley,44.0,4,11.00
2,FIN,Finland,1960,Squaw Valley,48.0,8,6.00
3,FRA,France,1960,Squaw Valley,26.0,3,8.67
4,GER,Germany,1960,Squaw Valley,74.0,8,9.25
...,...,...,...,...,...,...,...
217,SWE,Sweden,2018,PyeongChang,116.0,14,8.29
218,KOR,South Korea,2018,PyeongChang,122.0,17,7.18
219,JPN,Japan,2018,PyeongChang,124.0,13,9.54
220,CHN,People's Republic of China,2018,PyeongChang,80.0,9,8.89


In [12]:
df_final = df_final.to_csv("merged_olympics_final.csv", index=False)